# Calibracion de cámaras 

Para la captura de la base de datos se utilizaron dos cámaras **GoPro Hero3+ black edition**. Estas camaras son conocidas por su lente muy abierta, estilo "fisheye", la cual presenta una distorsion radial importante en sus capturas.

Resulta de extrema importancia entonces la calibracion, ya que se quieren capturar segmentos rectos (por ejemplo en el caso de los palos), y es entonces deseable reducir la distorsion radial antes mencionada.

## Primer intento
Debido a la "fama" de las camaras GoPro como camaras con lente "fisheye", se propuso utilizar la calibracion de OpenCV diseñada especialmente para estos tipos de lentes, donde la calibracion tradicional no suele arrojar buenos resultados.

Una vez implementada esta calibracion se observo que para la base de datos de capturas obtenidas para calibracion y los parametros obtenidos, la correccion de imagen era muy mala, incluso empeorando la distorsion radial.

Se llego entonces a la conclusion que esta calibracion es para lentes mas abiertos que los de las camaras utilizadas (cerca de los 180 grados).

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread, imsave
from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from scipy import signal
from scipy import misc
import scipy.ndimage.filters as filters
import scipy.ndimage as ndimage

import math
import time
import pdb
import os
import cv2
import skimage
#import imutils

assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import glob

CHECKERBOARD = (3,3)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.images = glob.glob('*.jpg')

images = glob.glob('./fotostestCalib/lowcut/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."    
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD,None)
    # If found, add object points, image points (after refining them)

    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners.reshape(1,-1,2))
        print(fname)
        
        
        
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

AssertionError: The fisheye module requires opencv version >= 3.0.0

In [ ]:
DIM=(640, 480)
K=np.array([[-95355.22890617317, -0.0, 324.14103837551085], [0.0, -144754.5038914807, 221.35289448332375], [0.0, 0.0, 1.0]])
D=np.array([[85705.42090888006], [-22734782.271395143], [1318406002.6250708], [260985563381.83984]])


img = cv2.imread('./fotostestCalib/lowcut/GOPR0623_resized.jpg')
h,w = img.shape[:2] 
    
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    
cv2.imshow("undistorted", undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Codigo basado en la implementacion de Kenneth Jiang

## Segundo intento

Luego de los malos resultados obtenidos con la calibracion fisheye, se implemento una variacion sobre el codigo utilizado para las entregas anteriores del curso de tratamiento de imagenes por computadora.

Obteniendo muy buenos resultados visuales, asi como un valor bajo del error de re-proyeccion calculado para el curso. 

Los resultados se observaron para un video en particular, el cual presentaba bastante distorsion, y eligiendo los parametros que arrojaban mejor resultad. Luego se procedió a aplicar la correccion a toda la base de datos a traves de un script que recorria todos los archivos.

In [ ]:
%matplotlib
import numpy as np
import cv2

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

WIDTH = 5  # esquinas del damero en la horizontal
HEIGHT = 4 # esquinas del damero en la vertical

# Modelo del damero en 3D
esquinasDamero3D = np.zeros((WIDTH*HEIGHT,3), np.float32)
esquinasDamero3D[:,:2] = np.mgrid[0:HEIGHT,0:WIDTH].T.reshape(-1,2)

## Listas que contienen los puntos que se utilizan para calibrar
puntosCalib3D = [] # puntos 3d a utilizar en la calibración
puntosCalib2D = [] # puntos 2d a utilizar en la calibración

buenas=0
images = glob.glob('./fotostestCalib/lowcut/*.jpg')
for fname in images:
    frame = cv2.imread(fname)


        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    ret, corners = cv2.findChessboardCorners(gray, (HEIGHT,WIDTH),None)
    
    # Si se encuentran las esquinas se refina su ubicación
    if ret == True:
            buenas+=1
            print(fname)
            cornersSubPixel = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
           
            # Se dibujan y muestran las esquinas detectadas
            frame = cv2.drawChessboardCorners(frame, (HEIGHT, WIDTH), cornersSubPixel,ret)
            
            cv2.imshow('captura',frame)
            
            puntosCalib3D.append(esquinasDamero3D)
            puntosCalib2D.append(cornersSubPixel)

    
ret, calibrationMatrix, distortionCoeff, rvecs, tvecs = cv2.calibrateCamera(puntosCalib3D, puntosCalib2D, gray.shape[::-1],None,None)
np.save('calibrationMatrix',calibrationMatrix )
np.save('distortionCoeff',distortionCoeff )
print("Calibracion terminada utilizando ", buenas, " fotos de 65 tomadas")
        
# Desplegar el frame. Terminar cuando se apriete la tecla 'q'
cv2.imshow('captura',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
    


In [8]:
matrizIntrinsecos = np.load('calibrationMatrix.npy')
coeficientesDistorsion = np.load('distortionCoeff.npy')

img = cv2.imread('./fotostestCalib/lowcut/GOPR0627_resized.jpg')
    
# se corrige la distorsión
imagenCorregida = cv2.undistort(img, matrizIntrinsecos ,coeficientesDistorsion)
        
        
# se muestra la imagen corregida
if imagenCorregida.shape[0]*imagenCorregida.shape[1] > 0:
    cv2.imwrite( "./corregidaa.jpg", imagenCorregida )
    cv2.imshow('captura corregida',imagenCorregida)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
#ERROR DE REPROYECCION
mediaerror = 0

for i in range(len(puntosCalib3D)):
    puntosCalib2D_aux, _ = cv2.projectPoints(puntosCalib3D[i], rvecs[i], tvecs[i], calibrationMatrix, distortionCoeff)
    
    error = cv2.norm(puntosCalib2D[i], puntosCalib2D_aux, cv2.NORM_L2)/len(puntosCalib2D_aux)
    mediaerror += error
    
reproerror=mediaerror/len(puntosCalib3D)    
print( "Error de reproyeccion: "+str(reproerror) )

NameError: name 'puntosCalib3D' is not defined

In [10]:
#Undistort video test
matrizIntrinsecos = np.load('calibrationMatrix.npy')
coeficientesDistorsion = np.load('distortionCoeff.npy')

cap = cv2.VideoCapture('./fotostestCalib/GOPR0548.MP4')
vid_writer = cv2.VideoWriter('./fotostestCalib/GOPR0548_undistroted.MP4',cv2.VideoWriter_fourcc('M','J','P','G'), 240, (848,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    
    # se corrige la distorsión
    imagenCorregida = cv2.undistort(frame, matrizIntrinsecos, coeficientesDistorsion)
        
        
    # se muestra la imagen corregida
    if imagenCorregida.shape[0]*imagenCorregida.shape[1] > 0:
        cv2.imshow('captura corregida',imagenCorregida)
        cv2.imshow('captura',frame)
        vid_writer.write(imagenCorregida)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
    

vid_writer.release()
cap.release()    
cv2.destroyAllWindows()

In [8]:
#Undistort videos
matrizIntrinsecos = np.load('calibrationMatrix.npy')
coeficientesDistorsion = np.load('distortionCoeff.npy')


videos = glob.glob('./gopro2/Rito/*.MP4')

for fname in videos:
    cap = cv2.VideoCapture(fname)
    print(fname)
    vid_writer = cv2.VideoWriter(os.path.dirname(fname)+'/undistorted/undist_'+os.path.basename(fname),cv2.VideoWriter_fourcc('M','J','P','G'), 240, (848,480))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
                
            # se corrige la distorsión
            imagenCorregida = cv2.undistort(frame, matrizIntrinsecos, coeficientesDistorsion)
        
        
            # se muestra la imagen corregida
            if imagenCorregida.shape[0]*imagenCorregida.shape[1] > 0:
                vid_writer.write(imagenCorregida)
        else:
            break
        
    vid_writer.release()
    cap.release()

./gopro2/Rito/GOPR8982.MP4
./gopro2/Rito/GOPR9000.MP4
./gopro2/Rito/GOPR8999.MP4
./gopro2/Rito/GOPR8977.MP4
./gopro2/Rito/GOPR8976.MP4
./gopro2/Rito/GOPR8983.MP4
./gopro2/Rito/GOPR8988.MP4
./gopro2/Rito/GOPR8986.MP4
./gopro2/Rito/GOPR9002.MP4
./gopro2/Rito/GOPR8987.MP4
./gopro2/Rito/GOPR8974.MP4
./gopro2/Rito/GOPR8997.MP4
./gopro2/Rito/GOPR8975.MP4
./gopro2/Rito/GOPR8998.MP4
./gopro2/Rito/GOPR8980.MP4
./gopro2/Rito/GOPR8979.MP4
./gopro2/Rito/GOPR8985.MP4
./gopro2/Rito/GOPR8981.MP4
./gopro2/Rito/GOPR8984.MP4
./gopro2/Rito/GOPR8978.MP4
./gopro2/Rito/GOPR8973.MP4
./gopro2/Rito/GOPR9001.MP4
